# Image-Catalog Interactivity

In [ ]:
# General python imports
import numpy as np

# Astropy
from astropy.visualization import  ZScaleInterval, AsinhStretch

# LSST imports
from lsst.daf.butler import Butler
import lsst.geom as geom

# Bokeh and Holoviews for visualization
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import rasterize

import panel as pn

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

## Load Data

In [ ]:
# Instantiate the Butler initializing it with the repository name and the DP0.1 collection identifier
#from lsst.daf.butler import Butler
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

In [ ]:
# Define a calibrated exposure and retrieve it via the Butler
calexpId = {'visit': 192350, 'detector': 175, 'band': 'i'}
calexp = butler.get('calexp', **calexpId)
assert calexp is not None
# Source table for this exposure
calexpSrc = butler.get('src', **calexpId)

In [ ]:
# Define a deep coadded image and retrieve it via the Butler
coaddId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd = butler.get('deepCoadd', **coaddId)
assert coadd is not None
# Source table for this coadd
coaddSrc = butler.get('deepCoadd_forced_src', coaddId)

## Example 1

In [ ]:
# Apply a asinh/zscale mapping to the data 
transform = AsinhStretch() + ZScaleInterval()
scaledImage = transform(calexp.image.array)

scaledImage = np.flipud(scaledImage)
bounds_img = (0, 0, calexp.getDimensions()[0], calexp.getDimensions()[1])

s = calexpSrc.getColumnView()
coords = s.getX(), s.getY()
detections = hv.Points(coords).opts(fill_color=None, size = 9, color="darkorange")

def makeImage(zmin=0.6, zmax=0.8):
    image = hv.Image(scaledImage, 
                     bounds=bounds_img, 
                     vdims=hv.Dimension('z', range=(zmin, zmax))).opts(
        cmap = "Greys_r", colorbar=True, height=600, width=700)
    #return rasterize(image, dynamic=False)
    return image

zmin = pn.widgets.FloatSlider(name='zmin', value=0.5, start=0., end=1., step=0.01)
zmax = pn.widgets.FloatSlider(name='zmax', value=1., start=0., end=1., step=0.01)

dmap = hv.DynamicMap(pn.bind(makeImage, zmin=zmin, zmax=zmax))

selection = streams.Selection1D(source=detections)

app = pn.Row(pn.WidgetBox('zmax', zmin, zmax), rasterize(dmap) * detections.opts(tools=['hover', 'box_select', 'lasso_select']))
app

> STOP - Select some data points from the plot above using the lasso or box select tool. Notice that the selection updates.

In [ ]:
selection

## Example 2

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              immediate=True, dataId=coaddId)

    return cutout_image

In [ ]:
source = ColumnDataSource(data=dict(ra=np.degrees(coaddSrc['coord_ra']),
                                    dec=np.degrees(coaddSrc['coord_dec']),
                                    flux=coaddSrc['modelfit_CModel_instFlux'],
                                    fluxErr=coaddSrc['modelfit_CModel_instFluxErr'],
                                    objectId=coaddSrc['id']))
detections = hv.Points(kdims=['flux', 'fluxErr'], vdims=['ra', 'dec', 'objectId'], data=source.data).opts(fill_color=None, size=9, color="darkorange")
detections.opts(tools=['tap'])
selection = streams.Selection1D(source=detections)

def makeImageCutout(index):
    if len(index) == 0:
        index = [0]
    
    ra = detections.data['ra'][index].values[0]
    dec = detections.data['dec'][index].values[0]
    objectId = detections.data['objectId'][index].values[0]
    
    data = cutout_coadd(butler, ra, dec)
    bounds_img = (0, 0, data.getDimensions()[0], data.getDimensions()[1])
    
    #transform = AsinhStretch()
    
    image = hv.Image(data.image.array, bounds=bounds_img).opts(
        cmap = "Greys_r", colorbar=True, height=600, width=700)
    
    return image.relabel("(RA, Dec) = (%.6f, %.6f) ; objectId = %i"%(ra, dec, objectId))

dmap = hv.DynamicMap(makeImageCutout, streams=[selection])

app = pn.Row(detections.opts(axiswise=True, height=600, width=600, logx=True, logy=True, xlim=(1, 1.e5),
                             nonselection_fill_color='silver', nonselection_line_color='silver'), 
             rasterize(dmap).opts(axiswise=True))
app